## News Sentiment

In [ ]:
pip install newsapi-python

# Manage API Key & Use NewsApiClient
I've saved my API key in a config file, I've loaded the file and saved my key to a variable.

In [37]:
import config

APIKEY=config.APIKEY

from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=APIKEY)

# What are the available sources?

In [30]:
import pandas as pd
sources = newsapi.get_sources()
print('How many total sources are available?:',len(sources['sources']))
total_sources = len(sources['sources'])
sources['sources'][0]

id=pd.Series(name='id')
name=pd.Series(name='name')
description=pd.Series(name='description')
url=pd.Series(name='url')
category=pd.Series(name='category')
language=pd.Series(name='language')
country=pd.Series(name='country')
#column_names = ['id', 'name', 'description', 'url', 'category', 'language', 'country']

#sourcedf=pd.DataFrame(columns=column_names)

for idx,outlet in enumerate(sources['sources']):
    id[idx]=outlet['id']
    name[idx] = outlet['name']
    description[idx] = outlet['description']
    url[idx] = outlet['url']
    category[idx] = outlet['category']
    language[idx] = outlet['language']
    country[idx] = outlet['country']
sourcedf=pd.concat([id,name,description,url,category,language,country],axis=1)

sourcedf['category'].unique()

How many total sources are available?: 128


array(['general', 'business', 'technology', 'sports', 'entertainment',
       'health', 'science'], dtype=object)

# Filter the available sources
Requirements: English Language, US Country

In [35]:
sourcedf= sourcedf[(sourcedf['language']=='en') & (sourcedf['country']=='us')]
sourcedf

,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
3,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",http://www.aljazeera.com,general,en,us
6,ars-technica,Ars Technica,The PC enthusiast's resource. Power users and ...,http://arstechnica.com,technology,en,us
8,associated-press,Associated Press,The AP delivers in-depth coverage on the inter...,https://apnews.com/,general,en,us
10,axios,Axios,Axios are a new media company delivering vital...,https://www.axios.com,general,en,us
15,bleacher-report,Bleacher Report,"Sports journalists and bloggers covering NFL, ...",http://www.bleacherreport.com,sports,en,us
16,bloomberg,Bloomberg,"Bloomberg delivers business and markets news, ...",http://www.bloomberg.com,business,en,us
17,breitbart-news,Breitbart News,Syndicated news and opinion website providing ...,http://www.breitbart.com,general,en,us
18,business-insider,Business Insider,Business Insider is a fast-growing business si...,http://www.businessinsider.com,business,en,us
20,buzzfeed,Buzzfeed,"BuzzFeed is a cross-platform, global network f...",https://www.buzzfeed.com,entertainment,en,us


In [43]:
# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q='',
                                          sources='fortune')
top_headlines

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'fortune', 'name': 'Fortune'},
   'author': 'Emma Hinchliffe, Paige McGlauflin',
   'title': 'Why a former SoftBank partner is tackling mid-career drop-off for working mothers',
   'description': 'Former SoftBank partner and Facebook India director Kirthiga Reddy is the cofounder of Laddrr, a resource hub for working mothers aiming to prevent mid-career drop-off.',
   'url': 'https://fortune.com/2022/06/01/former-softbank-partner-tackling-mid-career-drop-off-for-working-mothers/',
   'urlToImage': 'https://content.fortune.com/wp-content/uploads/2022/05/Kirthiga1.jpg?resize=1200,600',
   'publishedAt': '2022-06-01T13:22:34Z',
   'content': 'Skip to Content'},
  {'source': {'id': 'fortune', 'name': 'Fortune'},
   'author': 'Jessica Mathews',
   'title': 'Unprofitable companies needed to be profitable yesterday',
   'description': 'Capital is more expensive, and investors aren’t bullish on companies struggling to genera